In [ ]:
from langchain import hub
from dotenv import load_dotenv
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()

search = TavilySearchResults()
# query = "what is the weather in SF"
# search.invoke(query)
tools = [search]

prompt = hub.pull("hwchase17/openai-functions-agent")
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "whats the weather in sf?"})

In [6]:
from langchain.prompts import PromptTemplate 
from langchain.agents import initialize_agent                                   
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

search = TavilySearchResults()
tools = [search]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# PREFIX = '''You are an AI data scientist. You have done years of research in studying all the AI algorthims. 
# You also love to write. On free time you write blog post for articulating what you have learned about different 
# AI algorithms. Do not forget to include information on the algorithm's benefits, disadvantages, 
# and applications. Additionally, the blog post should explain how the algorithm advances model reasoning by a whopping 
# 70% and how it is a plug in and play version, connecting seamlessly to other components.
# '''

# FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:
# '''
# Thought: Do I need to use a tool? Yes
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# '''

# When you have gathered all the information regarding AI algorithm, just write it to the user in the form of a blog post.

# '''
# Thought: Do I need to use a tool? No
# AI: [write a blog post]
# '''
# """

# SUFFIX = '''

# Begin!

# Previous conversation history:
# {chat_history}

# Instructions: {input}
# {agent_scratchpad}
# '''


agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    memory=memory,
    # return_intermediate_steps=True,
    handle_parsing_errors=True,
    # agent_kwargs={
    #     'prefix': PREFIX, 
    #     'format_instructions': FORMAT_INSTRUCTIONS,
    #     'suffix': SUFFIX
    # }
)

# agent_executor = AgentExecutor(agent=agent, tools=tools)
agent.invoke({"input": "whats the temperature right now in Jaipur?"})




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: temperature in Jaipur right now
Observation: [{'url': 'https://www.timeanddate.com/weather/india/jaipur', 'content': 'Weather in Jaipur, Rajasthan, India. Time/General; Weather . Weather Today/Tomorrow ; Hour-by-Hour Forecast ... Now: 6:00 am: 7:00 am: 8:00 am: 9:00 am: 10:00 am: 79 °F: 77 °F: 77 °F: 77 °F: 79 °F: 81 °F: See more hour-by-hour weather. Forecast for the next 48 hours. Scroll right to see more Friday Saturday Night Morning Afternoon Evening ...'}, {'url': 'https://www.timeanddate.com/weather/india/jaipur/hourly', 'content': 'Hour-by-Hour Forecast for Jaipur, Rajasthan, India. Time/General. Time Zone. DST Changes. Sun & Moon. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 84 °F. Fog.'}, {'url': 'https://www.theweathernetwork.com/en/city/in/rajasthan/jaipur/current', 'content': 'Get Jaipur, RJ, IN c

{'input': 'whats the temperature right now in Jaipur?',
 'chat_history': [HumanMessage(content='whats the temperature right now in Jaipur?'),
  AIMessage(content='The temperature in Jaipur, India is currently 84°F.')],
 'output': 'The temperature in Jaipur, India is currently 84°F.'}

In [26]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate

# Define your custom prompt template with required variables
prompt = hub.pull("hwchase17/structured-chat-agent")

# Initialize the Google Gemini model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Set up Tavily as a search tool
tavily_tool = TavilySearchResults(max_results=10)

# Create the agent using your custom prompt template and Tavily tool
agent = create_react_agent(llm, tools=[tavily_tool], prompt=custom_prompt_template)

# Set up the agent executor
agent_executor = AgentExecutor(agent=agent, tools=[tavily_tool], verbose=True)

# Invoke the agent with a query
response = agent_executor.invoke({"input": "What is Python?"})
print(response)

c:\Users\Nilay Kumar\Desktop\langchain_app\venv\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


ValueError: Prompt missing required variables: {'agent_scratchpad', 'tools'}